In [1]:
import numpy as np

# 第２ステージ　自然なコードで表現する　

## ステップ１１　可変長の引数（順伝播編）

### 11.1 Functionクラスの修正
現状のFunctionクラスを、複数の入出力に対応できるよう修正する。<br>
以下、現状のFunctionクラス
```Python
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()
```

In [2]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)

In [3]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [4]:
class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]  # Get data from Variable
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]  # Wrap data

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


### 11.2 Addクラスの実装

In [5]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする

In [6]:
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs) # ysはタプル
y = ys[0]
print(y.data)

5


## ステップ１２　可変長の引数（改善編）

### 12.1 １つ目の改善：関数を使いやすく
まずは、関数を<strong>使う人</strong>にとっての改善を行う。<br>
- 現状のコード<br>
```Python
xs = [Variable(np.array(2)), Variable(np.array(3))]
f = Add()
ys = f(xs)
y = ys[0]
```
- 改善後のコード<br>
```Python
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)
```

In [7]:
class Function:
    def __call__(self, *inputs): # ①アスタリスクを付ける
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        # ②リストの要素が１つの時は最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

- memo<br>
<strong>可変長引数</strong><br>
変数にアスタリスクを付けることで、リストを使わずに任意の数の引数を与えて、<br>
関数を呼ぶことができる。以下に具体例を挙げる。

In [8]:
def print_x(*x):
    print(x)
    
print_x(1, 2, 3)

a, b = (1, 2), (3, 4)
print_x(a, b)

(1, 2, 3)
((1, 2), (3, 4))


In [9]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする

In [10]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

5


### 12.2 ２つ目の改善：関数を実装しやすく
今度は、関数を<strong>実装する人</strong>にとっての改善を行う。<br>
具体的には、メソッドの引数は直接受け取り、結果の変数も直接返す。<br>
- 現状のコード
```Python
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,) # 出力をタプルにする
```
- 改善後のコード
```Python
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
```

In [11]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs) # ①アスタリスクを付けてアンパッキング
        if not isinstance(ys, tuple): # ②タプルではない場合の追加対応
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 12.3 add関数の実装
Addクラスを<strong>Pythonの関数</strong>として利用できるようにする。

In [12]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

In [13]:
def add(x0, x1):
    return Add()(x0, x1)

In [14]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = add(x0, x1)
print(y.data)

5


## ステップ１３　可変長の引数（逆伝播編）

### 13.1 可変長引数に対応したAddクラスの逆伝播
ステップ１２の通り、足し算の順伝播は入力が２つ、出力が１つである。<br>
逆伝播時は、逆に、入力が１つ、出力が２つになる。<br>
例えば、$y=x_0+x_1$の場合、
$$\frac{\partial y}{\partial x_0} = 1, \frac{\partial y}{\partial x_1} = 1$$
であることから、足し算の逆伝播は、上流の微分を<strong>そのまま流す</strong>といいことがわかる。

In [15]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

### 13.2 Variableクラスの修正
- 現状のVariableクラス
```Py
class Variable:
    ...
    def backward(self):
    if self.grad is None:
        self.grad = np.ones_like(self.data)

    funcs = [self.creator]
    while funcs:
        f = funcs.pop()
        x, y = f.input, f.output # ①関数の入出力を取得 
        x.grad = f.backward(y.grad) # ②backwardメソッドを呼ぶ

        if x.creator is not None:
            funcs.append(x.creator)
```
現状では、①において、関数の入出力を１つだけに限定している。<br>
これを複数の変数に対応できるように4つの修正を加える。

In [16]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs] # ①
            gxs = f.backward(*gys) # ②
            if not isinstance(gxs, tuple): # ③
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs): # ④
                x.grad = gx

                if x.creator is not None:
                    funcs.append(x.creator)

①の箇所で、出力変数であるoutputsの微分をリストにまとめる。<br>
そして②において、関数fの逆伝播を呼び出す。ここで、変数に*をつけて、<br>
リストのアンパック（展開）を行う。<br>
また、③の箇所では、gxsがタプルではない時に、それをタプルへと変換する。
<br>
④では、各入力とそれに対応する微分のペアを設定する。


### 13.3 Squareクラスの実装

In [17]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data # 修正前はx = self.input.data
        gx = 2 * x * gy
        return gx

In [18]:
def square(x):
    f = Square()
    return f(x)

In [19]:
def add(x0, x1):
    return Add()(x0, x1)

In [20]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

13.0
4.0
6.0


- memo<br>
上記のプログラムでは、$z=x^2+y^2$という計算を行った。<br>
`z.backward()`の部分の計算過程を図にしたものが以下のものである。<br>
![fit13_3](./image/step13_ex.png)

## ステップ１４ 同じ変数を繰り返し使う
現状では、同じ変数を使って足し算を行うと、微分が正しく計算できない。<br>
そのため改善する。
以下の例では、$y=x+x$の時、$\frac{\partial y}{\partial x}=2$であるが、<br>
プログラムでは、1が出力されている。

In [21]:
x = Variable(np.array(3.0))
y = add(x, x)
print(f'y: {y.data}')

y.backward()
print(f'x.grad: {x.grad}')

y: 6.0
x.grad: 1.0


### 14.1 問題の原因
```Python
class Variable:
    ...
    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                x.grad = gx # ここが間違い！

                if x.creator is not None:
                    funcs.append(x.creator)
```
現状の実装では、同じ変数を繰り返し使った場合、伝播する微分の値が置き換わってしまう。<br>
正しくは、伝播する微分の<strong>和</strong>を求める必要がある。

### 14.2 解決策

In [22]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx # 初回の微分時にはそのまま代入
                else:
                    x.grad = x.grad + gx # 2回目以降では、微分を加算する

                if x.creator is not None:
                    funcs.append(x.creator)

In [23]:
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

2.0


In [24]:
x = Variable(np.array(3.0))
y = add(add(x, x), x)
y.backward()
print(x.grad)

3.0


### 14.3 微分をリセットする
これで解決かと思いきや、まだ注意すべき点がある。<br>
同じ変数を使って、別の計算を行う場合に、期待する動作をしない場合がある。

In [25]:
# １回目の計算
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# ２回目の計算（同じxを使って、別の計算を行う。）
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
5.0


In [26]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def cleargrad(self): # 微分をリセットするメソッドの追加
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    funcs.append(x.creator)

In [27]:
# １回目の計算
x = Variable(np.array(3.0))
y = add(x, x)
y.backward()
print(x.grad)

# ２回目の計算（同じxを使って、別の計算を行う。）
x.cleargrad() # 微分のリセット
y = add(add(x, x), x)
y.backward()
print(x.grad)

2.0
3.0


## ステップ１５ 複雑な計算グラフ（理論編）
省略

## ステップ16 複雑な計算グラフ（実装編）
### 16.1 世代の追加
インスタンス変数のgenerationで、どの<strong>世代</strong>の関数であるかを示す。

In [28]:
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs]) # 世代を決定する
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

### 16.2 世代順に取り出す
世代順に関数を取り出すためには、generationの値が大きい順に、関数を取り出せば良い。<br>
以下、ダミーの関数を使って、簡単な実験を行う。

In [29]:
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)
    
[f.generation for f in funcs]

[2, 0, 1, 4, 2]

毎回ソートするのは、計算の無駄があるのでは？<br>
世代の最大値をソートせずに抽出した方が速いのでは？<br>
と思ったので、簡単に比較してみる。

In [30]:
# 原著の実装法
funcs.sort(key=lambda x: x.generation)
print([f.generation for f in funcs])

f = funcs.pop()
f.generation

[0, 1, 2, 2, 4]


4

In [31]:
# sortせずに出してみる
generations = [2, 0, 1, 4, 2]
funcs = []

for g in generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)
    
f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
f.generation

4

どっちが速いか確かめてみる。<br>
リストの数によらず、sortしない方が速そう<br>
[このリンク](https://qiita.com/Hironsan/items/68161ee16b1c9d7b25fb)によると、データ数を$n$とした時、sort関数の計算量は$O(n \ log \ n)$、<br>
max関数の計算量は$O(n)$であるため、sortは使わない方が良さそう

In [117]:
import time
test_generations = np.random.randint(0, 100, 3).tolist()
funcs = []

for g in test_generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)

start = time.time()
funcs.sort(key=lambda x: x.generation)
#print([f.generation for f in funcs])

f = funcs.pop()
end = time.time() - start
print(f'f.generation: {f.generation}, time: {end: .6f}')

f.generation: 44, time:  0.000306


In [118]:
funcs = []

for g in test_generations:
    f = Function() # ダミーの関数クラス
    f.generation = g
    funcs.append(f)

start = time.time() 
f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
end = time.time() - start
print(f'f.generation: {f.generation}, time: {end: .6f}')

f.generation: 44, time:  0.000062


### 16.3 Variableクラスのbackward

In [34]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0 # 変数の宣言

    def set_creator(self, func):
        self.creator = func
        # 生みの親の関数を記憶するときに、親の関数より１大きい
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)

        add_func(self.creator)

        while funcs:
            # 世代の最大値をもつ関数を取得
            f = funcs.pop(funcs.index(max(funcs, key=lambda x:x.generation)))
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

### 16.4 動作確認 

In [35]:
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

$y=(x^2)^2 + (x^2)^2$の時、$y'=(2x^4)'=8x^3$より、$x=2.0$の時、微分は$64.0$となる。

In [36]:
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


## ステップ１７ メモリ管理と循環参照

### 17.2 参照カウント方式のメモリ管理

In [119]:
class obj:
    pass

def f(x):
    print(x)

In [121]:
a = obj() # 参照カウント１
f(a) # 関数の中では参照カウント２
# 関数を抜けると参照カウント１
a = None # 参照カウント０

In [122]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
a = b = c = None

### 17.3 循環参照

In [123]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c
c.a = a

a = b = c = None

### 17.4 weakrefモジュール

In [124]:
import weakref
import numpy as np

a = np.array([1, 2, 3])
b = weakref.ref(a)

b

<weakref at 0x7fbd09dbc9b0; to 'numpy.ndarray' at 0x7fbd09dbc6f0>

In [125]:
b()

array([1, 2, 3])

In [127]:
a = None
b # Jupyter形式では、ユーザの裏側で追加の参照を保持するので、インスタンスが消去されない

<weakref at 0x7fbd09dbc9b0; to 'numpy.ndarray' at 0x7fbd09dbc6f0>